# Importing Clinical Data

In [1]:
import os as os
import pickle as pickle
import pandas as pd

In [2]:
import Data as Data
from Helpers.Pandas import *

In [3]:
def get_run(firehose_dir, version='Latest'):
    '''
    Helper to get a run from the file-system. 
    '''
    path = '{}/ucsd_analyses'.format(firehose_dir)
    if version is 'Latest':
        version = sorted(os.listdir(path))[-1]
    run = pickle.load(open('{}/{}/RunObject.p'.format(path, version), 'rb'))
    return run

In [4]:
OUT_PATH = '/cellar/users/agross/TCGA_Code/CancerData/Data'
RUN_DATE = '2015_04_02'
VERSION = 'all'
CANCER = 'HNSC'
FIGDIR = '../Figures/'
if not os.path.isdir(FIGDIR):
    os.makedirs(FIGDIR)

In [5]:
run_path  = '{}/Firehose__{}/'.format(OUT_PATH, RUN_DATE)
run = get_run(run_path, 'Run_' + VERSION)
run.data_path = run_path 
run.result_path = run_path + 'ucsd_analyses'
run.report_path = run_path + 'ucsd_analyses/Run_all'

In [6]:
cancers = {c: run.load_cancer(c) for c in run.cancers if 
           os.path.isdir(run.report_path + '/' + c)}

In [7]:
from Processing.ProcessClinicalDataPortal import update_clinical_object

clinical = {c: cancer.load_clinical() for c, cancer in cancers.iteritems()}
for cancer,clin in clinical.iteritems():
    try:
        path  = '/cellar/users/agross/TCGA_Code/TCGA/Data/Followup_R10/' + cancer + '/'
        clinical[cancer] = update_clinical_object(clin, path)
    except:
        print cancer
        do_nothing = True

PCPG
UCS
TGCT
CHOL
THYM
MESO
FPPP


/cellar/users/agross/anaconda2/lib/python2.7/site-packages/Processing/ProcessClinicalDataPortal.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
  f['vitalstatus'] = f['daystodeath'].isnull()


In [12]:
codes = pd.concat([pd.Series(cancer, clinical[cancer].clinical.index)
                   for cancer in clinical])
codes = codes[codes != 'FPPP']
codes.name = 'codes'

In [9]:
surv = pd.DataFrame({c: v.survival.survival for c,v in clinical.iteritems()})
surv = surv.stack()

surv.index = surv.index.droplevel(2)
surv = surv.groupby(level=[0,1]).first()
surv = surv.ix[ti(surv[:,'days'] >= 7)]

surv_5y = pd.DataFrame({c: v.survival.survival_5y for c,v in clinical.iteritems()})
surv_5y = surv_5y.stack()
surv_5y.index = surv_5y.index.droplevel(2)
surv_5y = surv_5y.groupby(level=[0,1]).first()
surv_5y = surv_5y.ix[ti(surv[:,'days'] >= 7)]

In [10]:
for c in clinical.values():
    c = c.artificially_censor(10)
surv_10y = pd.DataFrame({c: v.survival.survival_10y for c,v in clinical.iteritems()})
surv_10y = surv_10y.stack()
surv_10y.index = surv_10y.index.droplevel(2)
surv_10y = surv_10y.groupby(level=[0,1]).first()
surv_10y = surv_10y.ix[ti(surv[:,'days'] >= 7)]

In [11]:
for c in clinical.values():
    c = c.artificially_censor(3)
surv_3y = pd.DataFrame({c: v.survival.survival_3y for c,v in clinical.iteritems()})
surv_3y = surv_3y.stack()
surv_3y.index = surv_3y.index.droplevel(2)
surv_3y = surv_3y.groupby(level=[0,1]).first()
surv_3y = surv_3y.ix[ti(surv[:,'days'] >= 7)]